In [1]:
import pandas as pd

import numpy as np
from scipy.stats import norm
import mean_confidence_interval as conf
import geometric_brownian_motion as gbm

import math
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
# Option parameters
S0 = 1
K = 1.1
T = 3
dt = 1
r = 0.06
n = 8

In [56]:
# Generate sample stock paths
paths = np.vstack([[1, 1.09, 1.08, 1.34],[1, 1.16, 1.26, 1.54],[1, 1.22, 1.07, 1.03],[1, 0.93, 0.97, 0.92],
           [1, 1.11, 1.56, 1.52],[1, 0.76, 0.77, 0.90],[1, 0.92, 0.84, 1.01],[1, 0.88, 1.22, 1.34]])
# paths = pd.DataFrame(paths,columns=['t=0','t=1','t=2','t=3'],)
payout = K - paths
payout[payout < 0] = 0
paths_rev = np.flip(paths,1)
payout_rev = np.flip(payout,1)
# paths_rev = paths.iloc[:, ::-1]
# payout_rev = payout.iloc[:, ::-1]
print (paths_rev)
print (payout_rev)

[[ 1.34  1.08  1.09  1.  ]
 [ 1.54  1.26  1.16  1.  ]
 [ 1.03  1.07  1.22  1.  ]
 [ 0.92  0.97  0.93  1.  ]
 [ 1.52  1.56  1.11  1.  ]
 [ 0.9   0.77  0.76  1.  ]
 [ 1.01  0.84  0.92  1.  ]
 [ 1.34  1.22  0.88  1.  ]]
[[ 0.    0.02  0.01  0.1 ]
 [ 0.    0.    0.    0.1 ]
 [ 0.07  0.03  0.    0.1 ]
 [ 0.18  0.13  0.17  0.1 ]
 [ 0.    0.    0.    0.1 ]
 [ 0.2   0.33  0.34  0.1 ]
 [ 0.09  0.26  0.18  0.1 ]
 [ 0.    0.    0.22  0.1 ]]


In [57]:
for i in range(payout_rev.shape[1]-1):
    payout_1 = payout_rev[:,i]
    payout_2 = payout_rev[:,i+1]
    
    print ('\n',payout_rev[:,i])
    print (payout_rev[:,i+1])
    
    # x - prices of stocks at timestep t, if non-zero payout at time t-1
    # paths - stock prices, .iloc[:,i+1] - limit to timestep t, .iloc[payout_2.nonzero()] - limit to nonzero t-1 payoffs 
    x = paths_rev[:,i+1][payout_2.nonzero()]
    # y - holding value from time t-1 to t
    HV = np.exp(-r)*payout_1[payout_2.nonzero()]
    
    # Fit quadratic regression
    c,b,a = np.polyfit(x,HV,2)

    # Find expected holding value based on regression
    E_HV = a + b * x + c * np.square(x)
    
    # Find Exercise value at time t-1
    EV = payout_2[payout_2.nonzero()]
    
    # overwrite payoffs at t and t-1
    # indexes of EV>E_HV
    for pos,ev in np.ndenumerate(EV):
        # if EV>E_HV, payout at t-1 is corresponding EV, and payout at t = 0
        print (pos)
        
        if ev>E_HV[pos]:
            print (payout_1[pos])
            payout_1[pos]=0
            payout_2[pos]=EV[pos]
        # if EV<E_HV, payout (value) at t-1 is corresponding HV
        else:
            payout_2[pos]=HV[pos]
    print ('\n',payout_rev[:,i])
    print (payout_rev[:,i+1])
    
    # Find cases where holding is optimal, and overwrite t-1 payout with discounted t HV
    for pos,p1 in np.ndenumerate(payout_1):
        print (p1,payout_2[pos])
        if p1>payout_2[pos]:
            payout_2[pos] = np.exp(-r)*payout_1[pos]
    
    print ('\n',payout_rev[:,i])
    print (payout_rev[:,i+1])
    break
print ('Option Value: ', np.mean(payout_rev[:,0]))


 [ 0.    0.    0.07  0.18  0.    0.2   0.09  0.  ]
[ 0.02  0.    0.03  0.13  0.    0.33  0.26  0.  ]
(0,)
(1,)
(2,)
0.07
(3,)
0.18
(4,)
0.0

 [ 0.    0.    0.    0.    0.    0.2   0.09  0.  ]
[ 0.          0.06592352  0.13        0.33        0.26        0.33        0.26
  0.        ]
0.0 0.0
0.0 0.0659235173509
0.0 0.13
0.0 0.33
0.0 0.26
0.2 0.33
0.09 0.26
0.0 0.0

 [ 0.    0.    0.    0.    0.    0.2   0.09  0.  ]
[ 0.          0.06592352  0.13        0.33        0.26        0.33        0.26
  0.        ]
Option Value:  0.03625


In [26]:
payout_rev

array([[ 0.        ,  0.        ,  0.        ,  0.29268374],
       [ 0.        ,  0.        ,  0.        ,  0.32019994],
       [ 0.        ,  0.        ,  0.        ,  0.29268374],
       [ 0.        ,  0.        ,  0.22      ,  0.32019994],
       [ 0.        ,  0.        ,  0.        ,  0.32019994],
       [ 0.2       ,  0.        ,  0.24485878,  0.23059931],
       [ 0.09      ,  0.        ,  0.22      ,  0.2071882 ],
       [ 0.        ,  0.        ,  0.22      ,  0.2071882 ]])

In [30]:
payout_rev[:,0]

array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.2 ,  0.09,  0.  ])